# BasicTick: Update HDB's Database Version
This notebook will add a changeset to remove a specific date partition.

In [1]:
HDB_CLUSTER_NAME="HDB_basictickdb_20230517_b"

# database
DB_NAME="basictickdb"

DATE_PARTS = ["2023.05.18"]

In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *


In [3]:
if DATE_PARTS is None:
    today_dt = datetime.datetime.today().strftime('%Y.%m.%d')
    DATE_PARTS = [ today_dt ]
    

In [4]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Database Details

In [5]:
dump_database(client, ENV_ID, DB_NAME)

Database: basictickdb, Changesets: 
Welcome kdb database
Bytes: 645,289,212 Changesets: 16 Files: 65
----------------------------------------------------------------------------------------------------


,changesetId,createdTimestamp,activeFromTimestamp,lastModifiedTimestamp,status
0,zsQYQc6biKkcJ80G1jpJoQ,2023-05-18 20:18:31.864000+00:00,2023-05-18 20:19:18.094000+00:00,2023-05-18 20:18:31.864000+00:00,COMPLETED
1,lsQYOcSS64hRVpFIYqEVgg,2023-05-18 20:00:58.150000+00:00,2023-05-18 20:00:59.783000+00:00,2023-05-18 20:00:58.150000+00:00,COMPLETED
2,DMQYIFB9kqe73jyPAo08Lw,2023-05-18 19:05:21.916000+00:00,2023-05-18 19:06:28.127000+00:00,2023-05-18 19:05:21.916000+00:00,COMPLETED
3,OMQVInp2DRl0j1X5NUur4Q,2023-05-17 15:12:22.253000+00:00,2023-05-17 15:12:24.751000+00:00,2023-05-17 15:12:22.253000+00:00,COMPLETED
4,vsQU4I81f69XJ8lC2SbqEw,2023-05-17 12:48:22.123000+00:00,2023-05-17 12:49:23.889000+00:00,2023-05-17 12:48:22.123000+00:00,COMPLETED
5,nMQS5Mzc85SJyrJYYKC2ZQ,2023-05-16 18:19:09.114000+00:00,2023-05-16 18:20:14.573000+00:00,2023-05-16 18:19:09.115000+00:00,COMPLETED
6,LsQScAK0eaCmSEsA4Ohfvg,2023-05-16 14:04:01.257000+00:00,2023-05-16 14:05:03.633000+00:00,2023-05-16 14:04:01.257000+00:00,COMPLETED
7,lMQSMf3l4oEm02YnETL0Rg,2023-05-16 11:48:32.332000+00:00,2023-05-16 11:48:52.528000+00:00,2023-05-16 11:48:32.332000+00:00,COMPLETED
8,8sQSMJXEyK6x7KlIK0UX7w,2023-05-16 11:45:27.946000+00:00,2023-05-16 11:46:10.285000+00:00,2023-05-16 11:45:27.946000+00:00,COMPLETED
9,9sQQP9DqbotzCLwXd8nd9w,2023-05-15 17:40:15.445000+00:00,2023-05-15 17:41:19.135000+00:00,2023-05-15 17:40:15.445000+00:00,COMPLETED


## (Virtually) Delete the Partition

In [7]:
changes=[]

for d in DATE_PARTS:
    changes.append( { 'changeType': 'DELETE', 'dbPath': f"/{d}/" } )
        
resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
    changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

Changeset...
{
    "changeRequests": [
        {
            "changeType": "DELETE",
            "dbPath": "/2023.05.18/"
        }
    ],
    "changesetId": "XMQYQtxvGvGI3XfsfAvfuw",
    "createdTimestamp": "2023-05-18 20:20:50.015000+00:00",
    "databaseName": "basictickdb",
    "environmentId": "ltb7ipj3j3kps24widyr4p",
    "lastModifiedTimestamp": "2023-05-18 20:20:50.015000+00:00",
    "status": "PENDING"
}


In [8]:
wait_for_changeset_status(client, ENV_ID, DB_NAME, changeset_id, show_wait=True)

Status is IN_PROGRESS, total wait 0 sec, waiting 10 sec ...


{'changesetId': 'XMQYQtxvGvGI3XfsfAvfuw',
 'databaseName': 'basictickdb',
 'environmentId': 'ltb7ipj3j3kps24widyr4p',
 'changeRequests': [{'changeType': 'DELETE', 'dbPath': '/2023.05.18/'}],
 'createdTimestamp': datetime.datetime(2023, 5, 18, 20, 20, 50, 15000, tzinfo=tzlocal()),
 'activeFromTimestamp': datetime.datetime(2023, 5, 18, 20, 20, 51, 547000, tzinfo=tzlocal()),
 'lastModifiedTimestamp': datetime.datetime(2023, 5, 18, 20, 20, 50, 15000, tzinfo=tzlocal()),
 'status': 'COMPLETED'}

In [9]:
DB_CONFIG=[{'databaseName': DB_NAME,
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': changeset_id}]

resp=client.update_kx_cluster_databases(environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, databases=DB_CONFIG)
resp

{'ResponseMetadata': {'RequestId': '8806748b-5d74-48b2-8b61-a408b211efdd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'date': 'Thu, 18 May 2023 20:21:03 GMT',
   'x-amzn-requestid': '8806748b-5d74-48b2-8b61-a408b211efdd',
   'x-amz-apigw-id': 'FIpLFEMuIAMFahg=',
   'x-amzn-trace-id': 'Root=1-646688ad-051d1e502ef672584e861cb0',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b3c18475b2db8cebe637fe45982c61a.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'wGhuhhefCd_lB9mQ2IAvjUvNZKqDXSKp1K-7dtn-Gwi_GwaYmUFzyA=='},
  'RetryAttempts': 0}}

In [10]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, show_wait=True)

Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 0 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 30 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 60 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 90 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 120 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 150 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 180 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 210 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 240 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING, total wait 270 sec, waiting 30 sec ...
Cluster: HDB_basictickdb_20230517_b status is UPDATING,

{'status': 'RUNNING',
 'clusterName': 'HDB_basictickdb_20230517_b',
 'clusterType': 'HDB',
 'port': 5000,
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': 'XMQYQtxvGvGI3XfsfAvfuw'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size': 1200}],
 'clusterDescription': 'Created with create_HDB for basic_tick notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.2xlarge', 'nodeCount': 3},
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0e702dec545865b11',
  'securityGroupIds': ['sg-018111774e795682d'],
  'subnetIds': ['subnet-05aa17ae202337a4e'],
  'ipAddressType': 'IP_V4'},
 'initializationScript': 'basictick/hdbmkdb.q',
 'commandLineArguments': [{'key': 's', 'value': '8'},
  {'key': 'dbdir', 'value': 'basictickdb'},
  {'key': 'codedir', 'value': 'basictick'}],
 'code': {'s3Bucket': 'kdb-demo-612841383594', 's3Key': 'code/basictick.zip'},
 'lastModifiedTimestamp': datetime.da

# Show Updated HDB
You can now return to the create_EOD_changset or query_HDB_updated notebooks to query the HDB after the update.

- [create_EOD_changeset](create_EOD_changeset.ipynb)
- [query_HDB_updated](query_HDB_updated.ipynb)

In [11]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-05-18 20:32:42.484481
